In [ ]:
import pandas as pd
print(pd.__version__)
from utils.utils import *
from utils.anthropic_utils import *

## Start here

In [ ]:
prompt_type = 'Metric'

name = "Batch_perturbation"
dataset = 'CSII'
model ='claude-3-haiku-20240307'

batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"
os.makedirs(batch_dir, exist_ok=True)
    
    
individual_data = load_data_metrics_for_prompt_short(dataset) 

In [ ]:
prompt_list = {}
for (hypo, hyper,tir), description in strings.items():
    for id in individual_data:

        pat = individual_data[id]
        comparator = description

        prompt_list[(hypo, hyper,tir,id)] = instructions(comparator,pat)

In [ ]:
i=0
save_dict = {}
for (hypo, hyper,tir,id), prompt in prompt_list.items():
    save_dict[i] = {
        "hypo": hypo,
        "hyper": hyper,
        "tir": tir,
        "id": id,
        "prompt": prompt
    }
    i+=1


# Load prompts

In [ ]:


batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"

batch_json, requests = make_batch_json(prompt_list, batch_dir , model)


save_json(
    data=requests,
    directory=batch_dir,
    custom_ending="requests.jsonl"
    )
simple_data = {
    'batch_id': batch_json.id,
    'status': batch_json.processing_status,
    'created_at': batch_json.created_at.isoformat() if batch_json.created_at else None,
    'saved_at': datetime.now().isoformat()
}
save_json(
    data=simple_data,
    directory=batch_dir,
    custom_ending="batch_sent.jsonl")



### Check on batch

In [ ]:

batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"

client = anthropic.Anthropic()

client.messages.batches.retrieve(
    load_json(batch_dir, "batch_sent.jsonl")['batch_id'],
)
